In [1]:
import numpy as np 
import pandas as pd 

In [2]:
import networkx as nx
import json
from collections import Counter
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot
from node2vec import Node2Vec
from datasketch import MinHash
import networkx as nx
import numpy as np
from sklearn.cluster import OPTICS

In [3]:
class TMinHash:
    def __init__(self, K, seed=1073):
        self.M = MinHash(num_perm=K, seed=seed)

    def fit(self, data):
        for sh in data:
            for p in sh.split("/"):
                self.M.update(p.encode('utf8'))

        return self.M
    
def load_graphs(filename):
    fid = open(filename, "r")
    call_graphs = fid.readlines()
    call_graphs_json = json.loads(call_graphs[0])
    behavior_graphs = list()

    for g in call_graphs_json:
        one_graph = nx.Graph()
        for node in g['call_graph']:
            # source node
            one_graph.add_node(node['toId'])  
            # destination node
            one_graph.add_node(node['fromId'])   
            # edge
            one_graph.add_edge(node['fromId'], node['toId'])
        behavior_graphs.append(one_graph)
    return behavior_graphs

def graph_to_minhash_key(graph: nx.Graph, K=2):
    edge_list = list(map(lambda x: x[0]+"_"+x[1], list(graph.edges)))
    tm = TMinHash(seed=1073, K=K)
    res = tm.fit(edge_list)
    key = "|".join(list(map(lambda x: str(x), sorted(res.hashvalues))))
    return key

def merge_graphs(master_graph: nx.Graph, graph_list: list):
    for g in graph_list:
        master_graph.add_nodes_from(g.nodes)
        master_graph.add_edges_from(g.edges)
    return master_graph

def semantically_related(api_id, api_id_name_dict, model, num_related=2):
    api_name = api_id_name_dict[api_id]
    related =  model.most_similar(api_id, topn=num_related)
    related_apis = list(map(lambda x: (x[0], api_id_name_dict[x[0]], x[1]), related))
    return (api_name, related_apis)

def graph2vec_edge_arithmetic(graph: nx.Graph, n2vmodel):
    
    sum_vec = None
    count = 0
    
    # for non trivial connected components
    for e in list(graph.edges):
        edgevec = np.multiply(n2vmodel.wv.get_vector(e[0]),n2vmodel.wv.get_vector(e[1]))
        norm = np.linalg.norm(edgevec)
        edgevec = edgevec if norm == 0 else edgevec/norm
        sum_vec = edgevec if sum_vec is None else np.add(sum_vec, edgevec)
        count += 1
    # getting all nodes that have a zero degree. These nodes a part of the trivial connected components
    node_degrees = list(map(lambda x: (x, graph.degree(x)), list(graph.nodes)))
    node_zero_degrees = list(filter(lambda x: x[1] == 0 , node_degrees))
    for (n, d) in node_zero_degrees:
        nodevec = n2vmodel.wv.get_vector(n)
        norm = np.linalg.norm(nodevec)
        nodevec = nodevec if norm == 0 else nodevec/norm
        sum_vec = nodevec if sum_vec is None else np.add(sum_vec, nodevec)
        count += 1
    try:
        graph_vector = sum_vec/count
        return graph_vector 
    except:
        print(sum_vec, count, len(graph.nodes), len(graph.edges))
        return None

def graph2vec_node_arithmetic(graph: nx.Graph, n2vmodel):
    
    sum_vec = None
    count = 0
    
    for n in list(graph.nodes):
        nodevec = n2vmodel.wv.get_vector(n)
        norm = np.linalg.norm(nodevec)
        nodevec = nodevec if norm == 0 else nodevec/norm
        sum_vec = nodevec if sum_vec is None else np.add(sum_vec, nodevec)
        count += 1
    try:
        graph_vector = sum_vec/count
        return graph_vector 
    except:
        print(sum_vec, count, len(graph.nodes), len(graph.edges))
        return None

In [4]:
supervised_graphs = load_graphs('supervised_call_graphs.json')
evaluation_graphs = load_graphs('remaining_call_graphs.json')

In [5]:
print('number of graphs in supervised datset: ', len(supervised_graphs))
print('number of graphs in evaluation datset: ', len(evaluation_graphs))

number of graphs in supervised datset:  1699
number of graphs in evaluation datset:  34423


In [6]:
graph_keys = list(map(lambda g: graph_to_minhash_key(g, K=2), supervised_graphs))
graph_hist = Counter(graph_keys)
print("supervised graphs")
print("Number of graphs: ",sum(list(map(lambda x: x[1], list(graph_hist.items())))), "Number of distinct graphs :", len(graph_hist))

graph_keys = list(map(lambda g: graph_to_minhash_key(g, K=2), evaluation_graphs))
graph_hist = Counter(graph_keys)
print("evaluation graphs")
print("Number of graphs: ",sum(list(map(lambda x: x[1], list(graph_hist.items())))), "Number of distinct graphs :", len(graph_hist))

supervised graphs
Number of graphs:  1699 Number of distinct graphs : 570
evaluation graphs
Number of graphs:  34423 Number of distinct graphs : 9064


In [7]:
the_graph = nx.Graph()
the_graph = merge_graphs(the_graph, supervised_graphs)
the_graph = merge_graphs(the_graph, evaluation_graphs)

In [8]:
node2vec = Node2Vec(the_graph, dimensions=20, walk_length=8, num_walks=100, workers=2)

Computing transition probabilities:   0%|          | 0/1640 [00:00<?, ?it/s]

In [9]:
model = node2vec.fit(window=4, min_count=0)

In [10]:

supervised_graphs_idx = list(zip(range(0, len(supervised_graphs)), supervised_graphs))

vector_list = list()
for (idx, g) in supervised_graphs_idx:
    graph_vector = graph2vec_edge_arithmetic(g, model)
    vector_list.append(graph_vector)

vector_list_edge_arithmetic = list(filter(lambda x: x is not None, vector_list))

None 0 0 0
None 0 0 0
None 0 0 0
None 0 0 0
None 0 0 0


In [11]:

supervised_graphs_idx = list(zip(range(0, len(supervised_graphs)), supervised_graphs))

vector_list = list()
for (idx, g) in supervised_graphs_idx:
    graph_vector = graph2vec_node_arithmetic(g, model)
    vector_list.append(graph_vector)

vector_list_node_arithmetic = list(filter(lambda x: x is not None, vector_list))

None 0 0 0
None 0 0 0
None 0 0 0
None 0 0 0
None 0 0 0


In [12]:

opt_edge = OPTICS()
opt_edge.fit(np.array(vector_list_edge_arithmetic))
opt_node = OPTICS()
opt_node.fit(np.array(vector_list_node_arithmetic))

/root/anaconda3/envs/exp1/lib/python3.6/site-packages/sklearn/cluster/_optics.py:807: RuntimeWarning:

divide by zero encountered in true_divide

/root/anaconda3/envs/exp1/lib/python3.6/site-packages/sklearn/cluster/_optics.py:807: RuntimeWarning:

divide by zero encountered in true_divide



OPTICS()

In [13]:
labels = list(opt_edge.labels_)
cluster_density = Counter(labels)
outlier_membership = cluster_density[-1]
number_of_clusters = len(cluster_density)
print("outlier membership: ", outlier_membership, "number of clusters: ", number_of_clusters)

outlier membership:  1246 number of clusters:  51


In [14]:
labels = list(opt_node.labels_)
cluster_density = Counter(labels)
outlier_membership = cluster_density[-1]
number_of_clusters = len(cluster_density)
print("outlier membership: ", outlier_membership, "number of clusters: ", number_of_clusters)

outlier membership:  1241 number of clusters:  60
